In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("MostPopularSuperhero4")
sc = SparkContext(conf = conf)

In [3]:
def extract_friends(super_hero_row):
    super_hero=super_hero_row.split()
    super_hero_id=int(super_hero[0])
    super_hero_friends=[int(sh_id) for sh_id in super_hero[1:]]
    return super_hero_id,super_hero_friends

superhero_friends_raw=sc.textFile("/home/ggomarr/Documents/Education/Udemy_Spark/23_Marvel-Graph.txt")
superhero_friends_count=(superhero_friends_raw.map(extract_friends)
                                              .map(lambda x: (x[0],len(x[1])))
                                              .reduceByKey(lambda x, y: x + y)
                                              .map(lambda x: (x[1],x[0])))

In [4]:
for superhero in superhero_friends_count.take(10):
    print('{:4d} - {:4d}'.format(superhero[0],superhero[1]))

   5 -    1
 122 -    2
  72 -    3
  14 -    4
   8 -    5
  21 -    6
   8 -    7
  15 -    8
  11 -    9
 187 -   10


In [8]:
max_friends_count_superhero=superhero_friends_count.max()

In [9]:
print('{:4d} - {:4d}'.format(max_friends_count_superhero[0],max_friends_count_superhero[1]))

1933 -  859


In [5]:
def process_superhero_row(superhero_row):
    import re
    superhero=superhero_row.split(' ',1)
    superhero_id=int(superhero[0])
    superhero_name=re.sub('^[^a-zA-Z0-9\s]*|[^a-zA-Z0-9\s]*$','',superhero[1].strip())
    return (superhero_id,superhero_name)

superhero_names_raw=sc.textFile("/home/ggomarr/Documents/Education/Udemy_Spark/23_Marvel-Names.txt")
superhero_names=superhero_names_raw.map(process_superhero_row)

In [6]:
for superhero in superhero_names.take(10):
    print('{:4d} - {}'.format(superhero[0],superhero[1]))

   1 - 24-HOUR MAN/EMMANUEL
   2 - 3-D MAN/CHARLES CHAN
   3 - 4-D MAN/MERCURIO
   4 - 8-BALL
   5 - A
   6 - A'YIN
   7 - ABBOTT, JACK
   8 - ABCISSA
   9 - ABEL
  10 - ABOMINATION/EMIL BLO


In [10]:
print('{:3d} - {:4d} - {}'.format(max_friends_count_superhero[0],
                                  max_friends_count_superhero[1],
                                  superhero_names.lookup(max_friends_count_superhero[1])[0]))

1933 -  859 - CAPTAIN AMERICA
